## Ucitavanje i Sredjivanje Nizova Podataka

In [1]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("serie-a-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("serie-a-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("serie-a-2025.csv")

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [3]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6.head(110)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
105,101,11,09/11/2025 11:30,Atleti Azzurri d'Italia,Atalanta,Sassuolo,0 - 3
106,102,11,09/11/2025 14:00,Renato Dall'Ara,Bologna,Napoli,2 - 0
107,104,11,09/11/2025 14:00,Luigi Ferraris,Genoa,Fiorentina,2 - 2
108,110,11,09/11/2025 17:00,Olimpico,Roma,Udinese,2 - 0
109,105,11,09/11/2025 19:45,Giuseppe Meazza,Inter,Lazio,2 - 0


In [4]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [5]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
105,101,11,09/11/2025 11:30,Atleti Azzurri d'Italia,Atalanta,Sassuolo,0 - 3
106,102,11,09/11/2025 14:00,Renato Dall'Ara,Bologna,Napoli,2 - 0
107,104,11,09/11/2025 14:00,Luigi Ferraris,Genoa,Fiorentina,2 - 2
108,110,11,09/11/2025 17:00,Olimpico,Roma,Udinese,2 - 0
109,105,11,09/11/2025 19:45,Giuseppe Meazza,Inter,Lazio,2 - 0


## Predprocesovanje Niza Podataka

In [6]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(870, 3)

In [7]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [8]:
df_BL

,H,A,Result
0,Empoli,Hellas Verona,0 - 1
1,Frosinone,Napoli,1 - 3
2,Genoa,Fiorentina,1 - 4
3,Inter,Monza,2 - 0
4,Roma,Salernitana,2 - 2
...,...,...,...
105,Atalanta,Sassuolo,0 - 3
106,Bologna,Napoli,2 - 0
107,Genoa,Fiorentina,2 - 2
108,Roma,Udinese,2 - 0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [10]:
df_domacin.columns

Index(['H_Bologna', 'H_Cagliari', 'H_Como', 'H_Cremonese', 'H_Empoli',
       'H_Fiorentina', 'H_Frosinone', 'H_Genoa', 'H_Hellas Verona', 'H_Inter',
       'H_Juventus', 'H_Lazio', 'H_Lecce', 'H_Milan', 'H_Monza', 'H_Napoli',
       'H_Parma', 'H_Pisa', 'H_Roma', 'H_Salernitana', 'H_Sassuolo',
       'H_Torino', 'H_Udinese', 'H_Venezia'],
      dtype='object')

In [11]:
df_domacin.shape

(870, 24)

In [12]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [13]:
df_gost.shape

(870, 24)

In [14]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
866,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [15]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [16]:
rez_dom.head()

0    0
1    1
2    1
3    2
4    2
Name: Result, dtype: object

In [17]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [18]:
rez_gost.head()

0    1
1    3
2    4
3    0
4    2
Name: Result, dtype: object

In [19]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [20]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski

,H,A,Result,Rez1,Rez2
0,Empoli,Hellas Verona,0 - 1,0,1
1,Frosinone,Napoli,1 - 3,1,3
2,Genoa,Fiorentina,1 - 4,1,4
3,Inter,Monza,2 - 0,2,0
4,Roma,Salernitana,2 - 2,2,2
...,...,...,...,...,...
105,Atalanta,Sassuolo,0 - 3,0,3
106,Bologna,Napoli,2 - 0,2,0
107,Genoa,Fiorentina,2 - 2,2,2
108,Roma,Udinese,2 - 0,2,0


In [21]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [22]:
#df_rezultatski[303:306] #sezona 23/4 od 0:306
#df_rezultatski[306:612] #sezona 24/5 od 306:612
#df_rezultatski[612:] #sezona25/6 od 612:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [23]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [24]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [25]:
#sezona 23/4
skor23_4 = skor[0:306]
skor24_5 = skor[306:612]
skor25_6 = skor[612:]

In [26]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_11984\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [27]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_11984\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [28]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [29]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [30]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [31]:
finalna_provjera.head()

,H,A,Rezultat,Skor
0,Empoli,Hellas Verona,0 - 1,-0.71
1,Frosinone,Napoli,1 - 3,-0.71
2,Genoa,Fiorentina,1 - 4,-0.71
3,Inter,Monza,2 - 0,0.71
4,Roma,Salernitana,2 - 2,0.00


# Konačni df - Podaci su Sređeni

In [32]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [33]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [34]:
df_BL.head(5)

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia,Skor
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00


In [35]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_bl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [36]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [37]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
604,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [38]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 4.20034012, -0.23202543, -0.17396309, ..., -0.22584665,
        -0.2163133 , -0.153393  ],
       [-0.23807596, -0.23202543, -0.17396309, ..., -0.22584665,
        -0.2163133 , -0.153393  ],
       [-0.23807596, -0.23202543, -0.17396309, ..., -0.22584665,
         4.62292424, -0.153393  ],
       ...,
       [-0.23807596, -0.23202543, -0.17396309, ..., -0.22584665,
        -0.2163133 , -0.153393  ],
       [-0.23807596, -0.23202543, -0.17396309, ..., -0.22584665,
        -0.2163133 ,  6.51920241],
       [-0.23807596, -0.23202543, -0.17396309, ..., -0.22584665,
        -0.2163133 , -0.153393  ]])

In [39]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [41]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [42]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                784       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [43]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [44]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
25/25 [==============================] - 6s 112ms/step - loss: 0.7902 - mae: 0.7902 - val_loss: 0.6109 - val_mae: 0.6109
Epoch 2/50
25/25 [==============================] - 1s 24ms/step - loss: 0.7411 - mae: 0.7411 - val_loss: 0.5925 - val_mae: 0.5925
Epoch 3/50
25/25 [==============================] - 1s 21ms/step - loss: 0.6963 - mae: 0.6963 - val_loss: 0.5797 - val_mae: 0.5797
Epoch 4/50
25/25 [==============================] - 0s 19ms/step - loss: 0.6547 - mae: 0.6547 - val_loss: 0.5737 - val_mae: 0.5737
Epoch 5/50
25/25 [==============================] - 1s 23ms/step - loss: 0.6171 - mae: 0.6171 - val_loss: 0.5787 - val_mae: 0.5787
Epoch 6/50
25/25 [==============================] - 1s 23ms/step - loss: 0.5841 - mae: 0.5841 - val_loss: 0.5817 - val_mae: 0.5817
Epoch 7/50
25/25 [==============================] - 1s 22ms/step - loss: 0.5585 - mae: 0.5585 - val_loss: 0.5843 - val_mae: 0.5843
Epoch 8/50
25/25 [==============================] - 1s 30ms/step - loss: 0.5400 - 

In [45]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.5737 - mae: 0.5737


In [46]:
#spremanje modela
model.save("sa_float_model.h5")

In [47]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.